<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-05/CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch

from google.colab import drive

drive.mount('/content/drive')

im_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256,256)),torchvision.transforms.ToTensor()])

train_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Train/Train', transform = im_transform)
train_loader = torch.utils.data.DataLoader(train_dat, batch_size = 32, shuffle = True)
valid_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Validation/Validation', transform = im_transform)
valid_loader = torch.utils.data.DataLoader(valid_dat, batch_size=len(valid_dat), shuffle = True)
test_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Test/Test', transform = im_transform)
test_loader = torch.utils.data.DataLoader(test_dat)


Mounted at /content/drive


In [2]:
from torch import nn

class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv_3_64 = nn.Conv2d(3, 64, kernel_size=(3,3), stride = 1, padding = 1)
    self.conv_64_64 = nn.Conv2d(64, 64, kernel_size=(3,3), stride = 1, padding = 1)
    self.pool = nn.MaxPool2d((2,2))
    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(64*64*64, 1024)
    self.fc2 = nn.Linear(1024, 128)
    self.fc3 = nn.Linear(128, 3)
    self.sm = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.pool(self.relu(self.conv_3_64(x)))
    x = self.pool(self.relu(self.conv_64_64(x)))
    x = torch.flatten(x, 1)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.relu(self.fc3(x))
    return self.sm(x)




In [5]:
from torch import optim
from sklearn.metrics import accuracy_score

epochs = 10
lr = 0.001
model = CNN()
model.to(torch.device('cuda'))
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr)

for e in range(epochs):
  model.train()
  losses = []
  for i, (x, y) in enumerate(train_loader):
    optimizer.zero_grad()
    x = x.to(torch.device('cuda'))
    y = y.to(torch.device('cuda'))
    pred = model(x)
    loss = loss_fn(pred, y)
    if i%10 == 0:
      print(loss)
    losses.append(loss.item())
    loss.backward()
  print("Epoch: {} | Loss: {:.4f}".format(e,np.mean(losses)))
  model.eval()
  with torch.no_grad():
    acc = 0
    for (x, y) in valid_loader:
      x = x.to(torch.device('cuda'))
      y = y.to(torch.device('cuda'))
      pred = model(x)
      loss = loss_fn(pred, y)
      print("Validation Accuracy {:.4f}".format(accuracy_score(torch.argmax(pred, 1), y)))

tensor(1.0997, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.0954, device='cuda:0', grad_fn=<NllLossBackward0>)


KeyboardInterrupt: 